In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
def dft(xn):
    '''
    xn is a column vector
    '''
    N = len(xn)
    xn = xn.astype(np.complex)
    D = np.ones(N, dtype=np.complex)
    for i in range(1, N):
        n = np.arange(N) * i
        arr = np.exp(-np.complex(0, 1) * 2 * np.pi * n / N)
        D = np.vstack([D, arr])
        #W = np.append(np.array(W), np.array(arr))
    X = np.matmul(D, xn)
    return X

In [3]:
def idft(X):#returns idft of X
    N = len(X)
    idft_matrix=np.empty([N,N],dtype=np.complex_)
    idft_matrix=np.matrix(idft_matrix)
    indices=np.arange(0,N,1)
    for k in indices:
        for n in indices:
            idft_matrix[k,n]=np.exp((1j)*2*np.pi*k*n/N)
    idft_matrix=idft_matrix/N
    xn = np.matmul(idft_matrix, X)
    return xn

In [4]:
def dct(xn):#returns dcft matrix
    N = len(xn)
    dct_matrix=np.empty([N,N],dtype=np.complex_)
    dct_matrix=np.matrix(dct_matrix)
    indices=np.arange(0,N,1)
    for k in indices:
        for n in indices:
            dct_matrix[k,n]=2*np.cos(np.pi*k*(2*n+1)/(2*N))
    
    X = np.matmul(dct_matrix, xn)
    return X

In [5]:
def idct(X):#returns dcft matrix
    N = len(X)
    idct_matrix=np.empty([N,N],dtype=np.complex_)
    idct_matrix=np.matrix(idct_matrix)
    indices=np.arange(0,N,1)
    for k in indices: 
        for n in indices:
            if n==0:
                alpha=0.5
            else:
                alpha=1
            idct_matrix[k,n]=alpha*np.cos(np.pi*n*(2*k+1)/(2*N))
    idct_matrix=idct_matrix/N
    xn = idct_matrix * X
    return xn

In [6]:
def haar(N):
    if N==2:
        h2=np.matrix([[1,1],[1,-1]])
        return h2
    v=np.log2(N)-1
    hn2=haar(N/2)
    result1=np.concatenate((hn2,hn2),axis=1)
    matrix1=(pow(2,v/2))*np.identity(int(N/2))
    result2=np.concatenate((matrix1,-matrix1),axis=1)
    return np.concatenate((result1,result2),axis=0)

In [7]:
def ihaar(N):
    haar_matrix=haar(N)
    return np.transpose(haar_matrix)/N

In [8]:
def generalised_transform(transform_type,x):
    N=len(x)
    #print(N)
    if transform_type==0:
        gtm=dft(x)
        return gtm
    if transform_type==1:
        gtm=dct(x)
        return gtm
    gtm=haar(N)
    return gtm*x

In [9]:
def generalised_inverse_transform(transform_type,X):
    N=len(X)
    #print(N)
    if transform_type==0:
        igtm=idft(X)
        return igtm
    if transform_type==1:
        igtm=idct(X)
        return igtm
    igtm=ihaar(N)
    return igtm*X 

In [10]:
def xmdft(X,L,N):
    for i in range(int((N+1-L)/2),int((N-1+L)/2)+1,1):
        X[i]=0
    return X

In [11]:
def  xdcthaar(X,L,N):
    for i in range(N-L,N):
        X[i]=0
    return X

In [18]:
def dot_product(x,y,N):
    err=x-y
    errt=np.transpose(err)
    errt=np.conjugate(errt)
    
    return errt*err/N

In [42]:
def mse(transform_type,x,X,N):
    err= np.array([])
    for l in range(0,N,1):
        if transform_type==0:
            Xm=xmdft(X,l,N)
            xm=generalised_inverse_transform(0,Xm)
        if transform_type==1:
            Xm=xdcthaar(X,l,N)
            xm=generalised_inverse_transform(1,Xm)
        if transform_type==2:
            Xm=xdcthaar(X,l,N)
            xm=generalised_inverse_transform(2,Xm)
        temp = np.mean(np.square(np.abs(x - xm)))
        err=np.append(err,temp)
    return err 

In [ ]:
# input sequence generator
length=100
real=np.random.rand(length)
img=np.random.rand(length)
x=real+1j*img
x=np.matrix(x)
x=np.transpose(x)
color=['r','g','b']
for i in range(0, 3, 1):
    X=generalised_transform(i,x)
    #print(X)
    err=mse(i,x,X,length)
    #print(err)
    plt.plot(np.arange(0,length,1),err,color[i])
